In [1]:
from datasets import load_dataset

In [2]:
ds = load_dataset("SWE-bench/SWE-bench_Verified")

In [5]:
ds['test'][0]

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-12907',
 'base_commit': 'd16bfe05a744909de4b27f5875fe0d4ed41ce607',
 'patch': "diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py\n--- a/astropy/modeling/separable.py\n+++ b/astropy/modeling/separable.py\n@@ -242,7 +242,7 @@ def _cstack(left, right):\n         cright = _coord_matrix(right, 'right', noutp)\n     else:\n         cright = np.zeros((noutp, right.shape[1]))\n-        cright[-right.shape[0]:, -right.shape[1]:] = 1\n+        cright[-right.shape[0]:, -right.shape[1]:] = right\n \n     return np.hstack([cleft, cright])\n \n",
 'test_patch': "diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py\n--- a/astropy/modeling/tests/test_separable.py\n+++ b/astropy/modeling/tests/test_separable.py\n@@ -28,6 +28,13 @@\n p1 = models.Polynomial1D(1, name='p1')\n \n \n+cm_4d_expected = (np.array([False, False, True, True]),\n+                  np.array([[True,  Tr

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://huggingface.co"  # 注释掉这行，因为它会导致 URL 构建失败

In [1]:
import pandas as pd

In [2]:
data = pd.read_parquet("/home/jiaxingliu/workspace/swe-pdb/rllm/rllm/data/datasets/SWE_Bench_Verified/test_verl.parquet")

In [5]:
data.iloc[0]

prompt               [{'content': 'placeholder', 'role': 'user'}]
reward_model              {'ground_truth': None, 'style': 'rule'}
extra_info      {'FAIL_TO_PASS': '["astropy/modeling/tests/tes...
Name: 0, dtype: object

In [8]:
data.iloc[0]['extra_info']

{'FAIL_TO_PASS': '["astropy/modeling/tests/test_separable.py::test_separable[compound_model6-result6]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model9-result9]"]',
 'PASS_TO_PASS': '["astropy/modeling/tests/test_separable.py::test_coord_matrix", "astropy/modeling/tests/test_separable.py::test_cdot", "astropy/modeling/tests/test_separable.py::test_cstack", "astropy/modeling/tests/test_separable.py::test_arith_oper", "astropy/modeling/tests/test_separable.py::test_separable[compound_model0-result0]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model1-result1]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model2-result2]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model3-result3]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model4-result4]", "astropy/modeling/tests/test_separable.py::test_separable[compound_model5-result5]", "astropy/modeling/tests/test_separable.py::t

In [4]:
os.environ['DOCKER_HOST'] = 'unix:///run/user/1001/podman/podman.sock'

In [14]:
from rllm.environments.swe.swe import SWEEnv
from datasets import load_dataset

In [19]:
from datasets import load_dataset
import os

# ❌ Incorrect Path (what you used):
# path_to_blob = "/data/jiaxingliu/cache/hf/hub/datasets--R2E-Gym--SWE-Bench-Verified/blobs/c63a1f1480c3c2cb0c12401098121d1d0a1210e779b5ea70f2cc931d5b512bad"

# ✅ Correct Path (from the snapshot directory):
base_path = '~/data/cache/hf/hub/datasets--R2E-Gym--SWE-Bench-Verified'
path_to_parquet = os.path.join(
    os.path.expanduser(base_path), 
    'snapshots/1fe83d7d3cb55a5eac714155f360614b3b7c2ad2/data/test-00000-of-00001.parquet'
)


# Load the dataset by specifying:
# 1. The file format: 'parquet'
# 2. The data_files dictionary with a split name ('test') and the correct path
ds = load_dataset('parquet', data_files={'test': path_to_parquet})

print(ds)

Generating test split: 500 examples [00:00, 577.01 examples/s]

DatasetDict({
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'parsed_commit', 'run_tests', 'docker_image'],
        num_rows: 500
    })
})


In [18]:
ds = load_dataset("/data/jiaxingliu/cache/hf/hub/datasets--R2E-Gym--SWE-Bench-Verified/blobs/c63a1f1480c3c2cb0c12401098121d1d0a1210e779b5ea70f2cc931d5b512bad")

FileNotFoundError: Couldn't find a dataset script at /data/jiaxingliu/cache/hf/hub/datasets--R2E-Gym--SWE-Bench-Verified/blobs/c63a1f1480c3c2cb0c12401098121d1d0a1210e779b5ea70f2cc931d5b512bad/c63a1f1480c3c2cb0c12401098121d1d0a1210e779b5ea70f2cc931d5b512bad.py or any data file in the same directory.

In [6]:
ds = load_dataset("R2E-Gym/R2E-Gym-Subset", split="train")


In [8]:
idx = 0

env = SWEEnv(entry=ds[idx], backend='docker', scaffold='r2egym')

In [10]:
observation, info = env.reset()


In [12]:
print(observation)


**Title:** Context migration fails to remove incompatible contexts, causing initialization errors

**Description:**
When initializing the `ContextHandler` with a mix of compatible and incompatible contexts, the migration process does not remove the incompatible contexts as expected. Instead, it raises an `IncompatibleContext` error, preventing successful initialization.

**Example Code:**
```python
handler = ContextHandler()
handler.bind(SimpleWidget)

widget = SimpleWidget()
contexts = [Context(foo=i) for i in (13, 13, 0, 1, 13, 2, 13)]

def migrate_context(context, _):
    if context.foo == 13:
        raise IncompatibleContext()

handler.initialize(widget, dict(context_settings=contexts))
# Expected: Incompatible contexts with foo=13 should be removed
# Actual: IncompatibleContext error is raised, and contexts are not removed
```

**Expected Behavior:**
During initialization, contexts that are incompatible (e.g., those that cause `IncompatibleContext` to be raised) should be automa

In [13]:
env.close()
